# Import Statements

In [1]:
import pandas as pd
import numpy as np
import sqlite3
from matplotlib import pyplot as plt
import dominate
from dominate.tags import *
import espn_api.football
from espn_api.football import League, Player
import time
import pickle
import uuid

from python import constants, functions

# Testing and Debugging

In [3]:
league = League(league_id=constants.LEAGUE_ID, year=2023, espn_s2=constants.ESPN_S2, swid=constants.SWID)

In [62]:
players = league.espn_request.get_pro_players()

players = players[281]
p = {
    'player_name':players['fullName'],
    'position':espn_api.football.constant.POSITION_MAP[players['defaultPositionId']],
    'pro_team':espn_api.football.constant.PRO_TEAM_MAP[players['proTeamId']]
}

p

{'player_name': 'Tony Pollard', 'position': 'RB', 'pro_team': 'TEN'}

In [61]:
for i in range(len(players)):
    if players[i]['fullName'] == 'Tony Pollard':
        print(i)


281


In [83]:
league = League(league_id=constants.LEAGUE_ID, year=2018, espn_s2=constants.ESPN_S2, swid=constants.SWID)

scoreboards = []

for week in range(1,16):
    scoreboards.append(
        {
            'Year':2018,
            'Week':week,
            'Scoreboard':league.scoreboard(week)
        }
    )

In [4]:
constants.YEARS_WEEKS

[(np.int64(2018), np.int64(13)),
 (np.int64(2019), np.int64(13)),
 (np.int64(2020), np.int64(13)),
 (np.int64(2021), np.int64(14)),
 (np.int64(2022), np.int64(14)),
 (np.int64(2023), np.int64(14)),
 (np.int64(2024), np.int64(14))]

In [ ]:
def summary_table(data: pd.DataFrame, year: int, week: int = None, averages: bool = False) -> pd.DataFrame:
    temp = data.loc[(data['Year'] == year) & (data['Playoff Flag'] == False)].copy()

    if week != None:
        temp = temp.loc[(data['Week'] <= week)]

    mean = temp['Score'].mean()
    std = temp['Score'].std() * 0.5

    opp_luck = []
    your_luck = []
    close_luck = []

    for your_score, opp_score in temp[['Score','Opp Score']].values:
        opp_luck.append(functions.opp_luck_score(opp_score=opp_score, mean=mean, std=std))
        your_luck.append(functions.your_luck_score(your_score=your_score, opp_score=opp_score, mean=mean, std=std))
        close_luck.append(functions.close_luck_score(your_score=your_score, opp_score=opp_score))

    temp['Opp Luck Score'] = opp_luck
    temp['Your Luck Score'] = your_luck
    temp['Close Luck Score'] = close_luck

    temp['Luck Score'] = temp[['Opp Luck Score','Your Luck Score','Close Luck Score']].sum(axis=1)

    temp_teams = temp['Team'].unique()
    weekly_standings = []

    for team in temp_teams:
        temp_team = temp.loc[temp['Team'] == team]
        wins = temp_team['Win'].sum()
        losses = temp_team['Win'].eq(0).sum()
        record = f'{wins}-{losses}'

        pf = round(temp_team['Score'].sum(), 2)
        avg_pf = round(pf / len(temp_team), 2)
        pa = round(temp_team['Opp Score'].sum(), 2)
        avg_margin = round((pf - pa) / len(temp_team), 2)
        luck_score = temp_team['Luck Score'].sum()

        weekly_standings.append(
            {
                'Team':team,
                'Wins':wins,
                'Record':record,
                'Points For':pf,
                'Points Against':pa,
                'Avg Points For':avg_pf,
                'Avg Margin':avg_margin,
                'Luck Score':luck_score
            }
        )
        # weekly_standings.append([team, wins, record, pf, pa, avg_pf, avg_margin, luck_score])

    

    # weekly_standings = pd.DataFrame(weekly_standings, columns=['Team','Wins','Record', 'Points For','Points Against','Avg Points For','Avg Margin','Luck Score'])
    weekly_standings = pd.DataFrame(weekly_standings)
    weekly_standings.sort_values(['Wins','Points For'], ascending=False, ignore_index=True, inplace=True)
    weekly_standings['Ranking'] = [i + 1 for i in weekly_standings.index]
    weekly_standings['Year'] = year
    weekly_standings = weekly_standings[['Year','Team','Record','Ranking','Points For','Points Against','Avg Points For','Avg Margin','Luck Score']]

    if averages:
        pf = round(temp['Score'].sum() / len(temp_teams), 2)
        pa = round(temp['Opp Score'].sum() / len(temp_teams), 2)
        luck_score = weekly_standings['Luck Score'].mean()

        avg_row = {
                'Year':'',
                'Team':'',
                'Record':'',
                'Ranking':'',
                'Points For':pf,
                'Points Against':pa,
                'Avg Points For':'',
                'Avg Margin':'',
                'Luck Score':''
        }
        weekly_standings.loc['Averages'] = avg_row

    return weekly_standings

In [36]:
df = summary_table(data=constants.GAME_DATA, year=2023, averages=True)
df

,Year,Team,Record,Ranking,Points For,Points Against,Avg Points For,Avg Margin,Luck Score
0,2023,Haris,12-2,1,1801.34,1456.30,128.67,24.65,10.0
1,2023,Carter,11-3,2,1787.82,1538.86,127.7,17.78,9.0
2,2023,Ethan,9-5,3,1574.82,1510.76,112.49,4.58,7.0
3,2023,Andrew,8-6,4,1672.62,1532.50,119.47,10.01,2.0
4,2023,Michael,8-6,5,1575.84,1666.08,112.56,-6.45,1.0
5,2023,Kevin,7-7,6,1800.40,1696.42,128.6,7.43,-8.0
6,2023,Noah,7-7,7,1579.60,1669.46,112.83,-6.42,-1.0
7,2023,Dante,6-8,8,1689.52,1667.06,120.68,1.6,-6.0
8,2023,Nathan,6-8,9,1496.26,1648.44,106.88,-10.87,4.0
9,2023,Zach,4-10,10,1488.32,1699.04,106.31,-15.05,-4.0
